In [5]:
options(repos = c(CRAN = "https://cloud.r-project.org"))

if (!require("pacman")) install.packages("pacman")
pacman::p_load(tidyverse, ggplot2, dplyr, lubridate, stringr, readxl, data.table, gdata, scales, data.table)

source("..//functions-1.R")
monthlist <- sprintf("%02d", 1:11)
y <- 2010

In [2]:
plan.data <- map_dfr(monthlist, ~ load_month(.x, y)) %>%
  arrange(contractid, planid, state, county, month) %>%
  group_by(state, county) %>%
  fill(fips, .direction = "downup") %>%                
  ungroup() %>%
  group_by(contractid, planid) %>%
  fill(plan_type, partd, snp, eghp, plan_name, .direction = "downup") %>%
  ungroup() %>%
  group_by(contractid) %>%
  fill(org_type, org_name, org_marketing_name, parent_org, .direction = "downup") %>%
  ungroup()

plan.data.dt <- as.data.table(plan.data)
setorder(plan.data.dt, contractid, planid, fips, year, month)
  
plan.year <- plan.data.dt[
, {
    nonmiss <- !is.na(enrollment)
    n <- sum(nonmiss)
    list(
      n_nonmiss = n,
      avg_enrollment = if (n>0) mean(enrollment[nonmiss]) else NA_real_,
      sd_enrollment  = if (n>1) sd(enrollment[nonmiss]) else NA_real_,
      min_enrollment = if (n>0) min(enrollment[nonmiss]) else NA_real_,
      max_enrollment = if (n>0) max(enrollment[nonmiss]) else NA_real_,
      first_enrollment = if (n>0) enrollment[which(nonmiss)[1]] else NA_real_,
      last_enrollment  = if (n>0) enrollment[tail(which(nonmiss), 1)] else NA_real_,
      state  = tail(state, 1),
      county = tail(county, 1),
      org_type = tail(org_type, 1),
      plan_type = tail(plan_type, 1),
      partd = tail(partd, 1),
      snp   = tail(snp, 1),
      eghp  = tail(eghp, 1),
      org_name = tail(org_name, 1),
      org_marketing_name = tail(org_marketing_name, 1),
      plan_name = tail(plan_name, 1),
      parent_org = tail(parent_org, 1),
      contract_date = tail(contract_date, 1)
    )
  },
by = .(contractid, planid, fips, year)
]

plan.data.2010 <- as_tibble(plan.year)

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or mo

In [6]:
service.year <- map_dfr(monthlist, ~ load_month_sa(.x, y))

# Ensure stable order before fills
service.year <- service.year %>%
  arrange(contractid, fips, state, county, month)

# Fill missing identifiers/labels
service.year <- service.year %>%
  group_by(state, county) %>%
  fill(fips, .direction = "downup") %>%
  ungroup() %>%
  group_by(contractid) %>%
  fill(plan_type, partial, eghp, org_type, org_name, .direction = "downup") %>%
  ungroup()

# Collapse to yearly: one row per contract × county (fips) × year --------
service.data.2010 <- service.year %>%
  group_by(contractid, fips, year) %>%
  arrange(month, .by_group = TRUE) %>%
  summarize(
    state     = last(state),
    county    = last(county),
    org_name  = last(org_name),
    org_type  = last(org_type),
    plan_type = last(plan_type),
    partial   = last(partial),
    eghp      = last(eghp),
    ssa       = last(ssa),
    notes     = last(notes),
    .groups = "drop"
  )

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or mo

In [7]:

# Import data -------------------------------------------------------------

ma.path.a <- paste0("../../ma-data/ma/landscape/Extracted Data/2010LandscapeSourceData_MA_12_01_09_A_to_M.csv")
ma.data.a <- read_csv(ma.path.a,
                      skip=5,
                      col_names=c("state","county","org_name","plan_name","plan_type","premium","partd_deductible",
                                  "drug_type","gap_coverage","drug_type_detail","demo_type","contractid",
                                  "planid","segmentid","moop"),
                      col_types = cols(
                        state = col_character(),
                        county = col_character(),
                        org_name = col_character(),
                        plan_name = col_character(),
                        plan_type = col_character(),
                        premium = col_number(),
                        partd_deductible = col_number(),
                        drug_type = col_character(),
                        gap_coverage = col_character(),
                        drug_type_detail = col_character(),
                        demo_type = col_character(),
                        contractid = col_character(),
                        planid = col_double(),
                        segmentid = col_double(),
                        moop = col_character()
                      ))


ma.path.b <- paste0("../../ma-data/ma/landscape/Extracted Data/2010LandscapeSourceData_MA_12_01_09_N_to_W.csv")
ma.data.b <- read_csv(ma.path.b,
                      skip=5,
                      col_names=c("state","county","org_name","plan_name","plan_type","premium","partd_deductible",
                                  "drug_type","gap_coverage","drug_type_detail","demo_type","contractid",
                                  "planid","segmentid","moop"),
                      col_types = cols(
                        state = col_character(),
                        county = col_character(),
                        org_name = col_character(),
                        plan_name = col_character(),
                        plan_type = col_character(),
                        premium = col_number(),
                        partd_deductible = col_number(),
                        drug_type = col_character(),
                        gap_coverage = col_character(),
                        drug_type_detail = col_character(),
                        demo_type = col_character(),
                        contractid = col_character(),
                        planid = col_double(),
                        segmentid = col_double(),
                        moop = col_character()
                      ))

ma.data <- rbind(ma.data.a,ma.data.b)


mapd.path.a <- paste0("../../ma-data/ma/landscape/Extracted Data/PartCD/2010/Medicare Part D 2010 Plan Report 09-14-09.xls")
mapd.data.a <- read_xls(mapd.path.a,
                        range="A5:AC26372",
                        sheet="Alabama to Montana",
                        col_names=c("state","county","org_name","plan_name","contractid","planid","segmentid",
                                    "org_type","plan_type","snp","snp_type","benefit_type","below_benchmark",
                                    "national_pdp","partd_rein_demo","partd_rein_demo_type","premium_partc",
                                    "premium_partd_basic","premium_partd_supp","premium_partd_total",
                                    "partd_assist_full","partd_assist_75","partd_assist_50","partd_assist_25",
                                    "partd_deductible","deductible_exclusions","increase_coverage_limit",
                                    "gap_coverage","gap_coverage_type"))


mapd.path.b <- paste0("../../ma-data/ma/landscape/Extracted Data/PartCD/2010/Medicare Part D 2010 Plan Report 09-14-09.xls")
mapd.data.b <- read_xls(mapd.path.b,
                        range="A5:AC31073",
                        sheet="Nebraska to Wyoming",
                        col_names=c("state","county","org_name","plan_name","contractid","planid","segmentid",
                                    "org_type","plan_type","snp","snp_type","benefit_type","below_benchmark",
                                    "national_pdp","partd_rein_demo","partd_rein_demo_type","premium_partc",
                                    "premium_partd_basic","premium_partd_supp","premium_partd_total",
                                    "partd_assist_full","partd_assist_75","partd_assist_50","partd_assist_25",
                                    "partd_deductible","deductible_exclusions","increase_coverage_limit",
                                    "gap_coverage","gap_coverage_type"))
mapd.data <- rbind(mapd.data.a,mapd.data.b)

landscape.2010 <- mapd.clean.merge(ma.data=ma.data, mapd.data=mapd.data, y)

Warning message:
“Expecting logical in AA1297 / R1297C27: got 'Y'”
Warning message:
“Expecting logical in AA1298 / R1298C27: got 'Y'”
Warning message:
“Expecting logical in AA1299 / R1299C27: got 'Y'”
Warning message:
“Expecting logical in AA1300 / R1300C27: got 'Y'”
Warning message:
“Expecting logical in AA3255 / R3255C27: got 'Y'”
Warning message:
“Expecting logical in AA3256 / R3256C27: got 'Y'”
Warning message:
“Expecting logical in AA3356 / R3356C27: got 'Y'”
Warning message:
“Expecting logical in AA3420 / R3420C27: got 'Y'”
Warning message:
“Expecting logical in AA3450 / R3450C27: got 'Y'”
Warning message:
“Expecting logical in AA3452 / R3452C27: got 'Y'”
Warning message:
“Expecting logical in AA3453 / R3453C27: got 'Y'”
Warning message:
“Expecting logical in AA3480 / R3480C27: got 'Y'”
Warning message:
“Expecting logical in AA3627 / R3627C27: got 'Y'”
Warning message:
“Expecting logical in AA3628 / R3628C27: got 'Y'”
Warning message:
“Expecting logical in AA3629 / R3629C27: got 

In [8]:
ma.penetration <- map_dfr(monthlist, ~ load_month_pen(.x, y)) %>%
  arrange(state, county, month) %>%
  group_by(state, county) %>%
  fill(fips, .direction = "downup") %>%
  ungroup()

# Collapse to yearly (safe summaries; avoid NaN/Inf) ----------------------
pen.2010 <- ma.penetration %>%
  group_by(fips, state, county, year) %>%
  arrange(month, .by_group = TRUE) %>%
  summarize(
    n_elig  = sum(!is.na(eligibles)),
    n_enrol = sum(!is.na(enrolled)),

    avg_eligibles   = ifelse(n_elig  > 0, mean(eligibles, na.rm = TRUE), NA_real_),
    sd_eligibles    = ifelse(n_elig  > 1,  sd(eligibles,  na.rm = TRUE), NA_real_),
    min_eligibles   = ifelse(n_elig  > 0, min(eligibles,  na.rm = TRUE), NA_real_),
    max_eligibles   = ifelse(n_elig  > 0, max(eligibles,  na.rm = TRUE), NA_real_),
    first_eligibles = ifelse(n_elig  > 0, first(na.omit(eligibles)),     NA_real_),
    last_eligibles  = ifelse(n_elig  > 0,  last(na.omit(eligibles)),     NA_real_),

    avg_enrolled    = ifelse(n_enrol > 0, mean(enrolled,   na.rm = TRUE), NA_real_),
    sd_enrolled     = ifelse(n_enrol > 1,  sd(enrolled,    na.rm = TRUE), NA_real_),
    min_enrolled    = ifelse(n_enrol > 0, min(enrolled,    na.rm = TRUE), NA_real_),
    max_enrolled    = ifelse(n_enrol > 0, max(enrolled,    na.rm = TRUE), NA_real_),
    first_enrolled  = ifelse(n_enrol > 0, first(na.omit(enrolled)),       NA_real_),
    last_enrolled   = ifelse(n_enrol > 0,  last(na.omit(enrolled)),       NA_real_),

    ssa = last(ssa),
    .groups = "drop"
  )

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”


In [9]:
## 2010 Rating variable names
rating.vars.2010=c("contractid",
                   "org_type",
                   "contract_name",
                   "org_marketing",
                   "breastcancer_screen",
                   "rectalcancer_screen",
                   "cv_diab_cholscreen",
                   "glaucoma_test",
                   "monitoring",
                   "flu_vaccine",
                   "pn_vaccine",
                   "physical_health",
                   "mental_health",
                   "osteo_test",
                   "physical_monitor",
                   "primaryaccess",
                   "osteo_manage",
                   "diab_healthy",
                   "bloodpressure",
                   "ra_manage",
                   "copd_test",
                   "bladder",
                   "falling",
                   "nodelays",
                   "doctor_communicate",                   
                   "carequickly",
                   "customer_service",                   
                   "overallrating_care",
                   "overallrating_plan",
                   "complaints_plan",
                   "appeals_timely",
                   "appeals_review",
                   "leave_plan",
                   "audit_problems",
                   "hold_times",
                   "info_accuracy",
                   "ttyt_available")


# Import data -------------------------------------------------------------

ma.path.a <- "../../ma-data/ma/star-ratings/Extracted Star Ratings/2010/2010_Part_C_Report_Card_Master_Table_2009_11_30_domain.csv"
star.data.a <- read_csv(
  ma.path.a,
  skip = 4,
  col_names = rating.vars.2010,
  na = c("", "NA", "*")
) %>%
  mutate(across(
    -any_of(c("contractid","org_type","contract_name","org_marketing")),
    ~ parse_number(as.character(.))
  ))


ma.path.b <- "../../ma-data/ma/star-ratings/Extracted Star Ratings/2010/2010_Part_C_Report_Card_Master_Table_2009_11_30_summary.csv"
star.data.b <- read_csv(
  ma.path.b,
  skip = 2,
  col_names = c("contractid","org_type","contract_name","org_marketing","partc_score"),
  na = c("", "NA", "*")
) %>%
  mutate(
    new_contract = ifelse(partc_score == "Plan too new to be measured", 1, 0),
    partc_score  = ifelse(new_contract == 1, NA_real_, parse_number(as.character(partc_score)))
  ) %>%
  select(contractid, new_contract, partc_score) %>%
  mutate(partcd_score = NA_real_)

final.star.ratings <- star.data.a %>%
  select(-contract_name, -org_type, -org_marketing) %>%  
  left_join(star.data.b, by=c("contractid")) %>%
  mutate(year=2010)

Rows: 709 Columns: 37
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (37): contractid, org_type, contract_name, org_marketing, breastcancer_s...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
“There were 33 warnings in `mutate()`.
The first warning was:
ℹ In argument: `across(...)`.
Caused by warning:
! 364 parsing failures.
row col expected                              actual
  1  -- a number Plan not required to report measure
  2  -- a number Plan not required to report measure
  3  -- a number Plan not required to report measure
  4  -- a number Plan too new to be measured        
  5  -- a number Plan too new to be measured        
... ... ........ ...................................
See problems(...) for more details.
ℹ Run `dplyr::last_dplyr_warnings()` to see the 32 remaining warnings.”
Rows: 709 Column

In [14]:
# Import data -------------------------------------------------------------

ma.path.a <- "../../ma-data/ma/cms-payment/2010/2010PartCPlanLevel2.xlsx"
risk.rebate.a <- read_xlsx(ma.path.a,range="A4:H3157",
                        col_names=c("contractid","planid","contract_name","plan_type",
                                    "riskscore_partc","payment_partc","rebate_partc",
                                    "msa_deposit_partc"))
ma.path.b <- "../../ma-data/ma/cms-payment/2010/2010PartDPlans2.xlsx"
risk.rebate.b <- read_xlsx(ma.path.b,range="A4:H4417",
                        col_names=c("contractid","planid","contract_name","plan_type",
                                    "directsubsidy_partd","riskscore_partd","reinsurance_partd",
                                    "costsharing_partd"))


risk.rebate.a <- risk.rebate.a %>%
  mutate(
    across(c(riskscore_partc, payment_partc, rebate_partc),
           ~ parse_number(as.character(.))),
    planid = as.numeric(planid),
    year   = 2010
  ) %>%
  select(contractid, planid, contract_name, plan_type,
         riskscore_partc, payment_partc, rebate_partc, year)    

risk.rebate.b <- risk.rebate.b %>%
  mutate(
    across(c(directsubsidy_partd, reinsurance_partd, costsharing_partd),
           ~ parse_number(as.character(.))),
    payment_partd = directsubsidy_partd + reinsurance_partd + costsharing_partd,
    planid        = as.numeric(planid)
  ) %>%
  select(contractid, planid, payment_partd,
         directsubsidy_partd, reinsurance_partd, costsharing_partd,
         riskscore_partd)  
  
rebate.2010 <- risk.rebate.a %>%
  left_join(risk.rebate.b, by=c("contractid","planid"))

In [15]:
# Import data -------------------------------------------------------------

ffs.data <- read_csv("../../ma-data/ffs-costs/Extracted Data/Aged Only/aged10.csv",
                  skip=7,
                  col_names=FALSE, na="*")

ffs.data <- ffs.data[,1:15]
names(ffs.data) <- c("ssa","state","county_name","parta_enroll",
                    "parta_reimb","parta_percap","parta_reimb_unadj",
                    "parta_percap_unadj","parta_ime","parta_dsh",
                    "parta_gme","partb_enroll",
                    "partb_reimb","partb_percap",
                    "mean_risk")  

final.ffs.costs <- ffs.data %>%
  select(ssa,state,county_name,parta_enroll,parta_reimb,
         partb_enroll,partb_reimb,mean_risk) %>%
  mutate(year=2010,
         ssa=as.numeric(ssa)) %>%
  mutate(across(c(parta_enroll, parta_reimb, partb_enroll, partb_reimb, mean_risk),
              ~ parse_number(as.character(.))))

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 3245 Columns: 15
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): X1, X2, X3
dbl (4): X6, X8, X14, X15
num (8): X4, X5, X7, X9, X10, X11, X12, X13

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [16]:
ma.2010 <- plan.data.2010 %>%
  inner_join(service.data.2010 %>% select(contractid, fips),
             by = c("contractid","fips")) %>%
  filter(!state %in% c("VI","PR","MP","GU","AS",""),
         snp == "No",
         (planid < 800 | planid >= 900),
         !is.na(planid), !is.na(fips)) %>%
  left_join(pen.2010 %>% ungroup() %>% rename(state_long = state, county_long = county) %>%
            mutate(state_long=str_to_lower(state_long)) %>% 
            group_by(fips) %>% mutate(ncount=n()) %>% filter(ncount==1),
            by = c("fips"))

state.2010 <- ma.2010 %>%
  group_by(state) %>%
  summarize(state_name = last(state_long[!is.na(state_long)]), .groups = "drop")

full.2010 <- ma.2010 %>%
  left_join(state.2010, by = "state") %>%
  left_join(landscape.2010 %>% mutate(state=str_to_lower(state)), by = c("contractid","planid","state_name" = "state","county")) %>%
  left_join(rebate.2010 %>% select(-contract_name, -plan_type), by = c("contractid","planid")) %>%
  left_join(final.star.ratings  %>% select(-year), by=c("contractid")) %>% mutate(
    basic_premium = case_when(
      rebate_partc > 0 ~ 0,
      partd == "No" & !is.na(premium) & is.na(premium_partc) ~ premium,
      TRUE ~ premium_partc
    ),
    bid = case_when(
      rebate_partc == 0 & basic_premium > 0 ~ (payment_partc + basic_premium) / riskscore_partc,
      rebate_partc > 0  | basic_premium == 0 ~  payment_partc / riskscore_partc,
      TRUE ~ NA_real_
    )
  )

In [17]:
full.2010 <- full.2010 %>%
  select(-year.y, -year.x.x, -year.y.y) %>%
  rename(year = year.x)

In [18]:
write_csv(full.2010,"../data/data-2010.csv")